In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "growing_degree_days_gt_4"

In [4]:
#Variable 
variable85="rcp85_"+var+"_seasonal"
variable45="rcp45_"+var+"_seasonal"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_growing_degree_days_gt_4_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/por

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [5]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    #print(tmp1)
    i = i+1
    listds.append(tmp1)

0
1
2
3


## Loop to create all 4 dataframes

In [6]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"],2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"],2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df45f.columns = [season + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [7]:
#df45list


In [8]:
dfw45 = df45list[0].reset_index()
dfsp45 = df45list[1].reset_index()
dfsu45 = df45list[2].reset_index()
dff45 = df45list[3].reset_index()

mkey = ["lat","lon"]
dfse45 = dfw45.merge(dfsp45, on=mkey).merge(dfsu45, on=mkey).merge(dff45, on=mkey)
dfse45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_seasonal.feather")

In [9]:
dfse452 = dfse45.drop(columns=["winter_hist_rcp45_p10","winter_hist_rcp45_p50", "winter_hist_rcp45_p90", 
                               "spring_hist_rcp45_p10", "spring_hist_rcp45_p50", "spring_hist_rcp45_p90", 
                               "summer_hist_rcp45_p10","summer_hist_rcp45_p50", "summer_hist_rcp45_p90", 
                               "fall_hist_rcp45_p10","fall_hist_rcp45_p50", "fall_hist_rcp45_p90"])
dfse452


,lat,lon,winter_t2050_rcp45_p10,winter_t2080_rcp45_p10,winter_t2050_rcp45_p50,winter_t2080_rcp45_p50,winter_t2050_rcp45_p90,winter_t2080_rcp45_p90,spring_t2050_rcp45_p10,spring_t2080_rcp45_p10,...,summer_t2050_rcp45_p50,summer_t2080_rcp45_p50,summer_t2050_rcp45_p90,summer_t2080_rcp45_p90,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90
0,40.041039,-89.045212,42.39,52.42,86.31,104.05,153.61,174.72,749.43,795.70,...,1971.89,2041.45,2100.00,2165.44,910.20,925.41,1032.10,1085.45,1164.08,1230.76
1,40.041039,-88.961884,42.03,52.12,85.77,103.43,152.76,173.48,747.35,793.61,...,1970.38,2040.29,2098.44,2163.63,910.01,925.28,1032.31,1085.44,1164.24,1230.68
2,40.041039,-88.878548,41.71,51.87,85.17,102.96,152.01,172.45,746.03,792.18,...,1970.27,2040.39,2098.01,2163.05,909.96,925.25,1032.50,1085.60,1164.42,1230.74
3,40.041039,-88.795219,40.91,51.25,84.13,101.80,150.25,170.62,743.60,789.52,...,1970.25,2040.29,2097.71,2162.78,908.31,923.69,1030.48,1083.51,1162.31,1228.63
4,40.041039,-88.711891,40.94,51.30,84.03,101.92,149.97,170.29,745.08,790.98,...,1972.60,2042.55,2099.37,2164.38,911.11,926.38,1032.70,1086.22,1164.37,1230.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,...,153.62,157.31,301.88,271.48,0.04,0.02,2.58,2.88,14.86,13.53
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,...,136.71,139.09,279.99,249.23,0.02,0.00,1.41,1.74,11.81,10.48
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,...,147.88,150.32,294.14,263.86,0.03,0.00,2.10,2.43,13.78,12.36
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,...,151.27,154.13,298.41,268.93,0.04,0.01,2.33,2.65,14.40,12.93


In [10]:
dfsew = dfse45[["winter_hist_rcp45_p50"]]
dfsesp = dfse45[["spring_hist_rcp45_p50"]]
dfsesu = dfse45[["summer_hist_rcp45_p50"]]
dfsefa = dfse45[["fall_hist_rcp45_p50"]]

# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [11]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * time                      (time) datetime64[ns] 1981-12-01 ... 2100-12-01
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset

## Loop to create all 4 dataframes

In [12]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"],2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"],2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"],2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df85f.columns = [season + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [13]:
df85list


[                      winter_hist_rcp85_p10  winter_t2050_rcp85_p10  \
 lat       lon                                                         
 40.041039 -89.045212                  14.79                   55.47   
           -88.961884                  14.71                   55.04   
           -88.878548                  14.60                   54.70   
           -88.795219                  14.29                   53.96   
           -88.711891                  14.34                   54.06   
 ...                                     ...                     ...   
 66.623306 -61.629639                   0.00                    0.00   
           -61.546310                   0.00                    0.00   
           -61.462978                   0.00                    0.00   
           -61.379650                   0.00                    0.00   
           -61.296322                   0.00                    0.00   
 
                       winter_t2080_rcp85_p10  winter_hist_rcp

In [14]:
dfw85 = df85list[0].reset_index()
dfsp85 = df85list[1].reset_index()
dfsu85 = df85list[2].reset_index()
dff85 = df85list[3].reset_index()
mkey = ["lat","lon"]
dfse85 = dfw85.merge(dfsp85, on=mkey).merge(dfsu85, on=mkey).merge(dff85, on=mkey)
dfse85

,lat,lon,winter_hist_rcp85_p10,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_hist_rcp85_p50,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_hist_rcp85_p90,winter_t2050_rcp85_p90,...,summer_t2080_rcp85_p90,fall_hist_rcp85_p10,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_hist_rcp85_p50,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_hist_rcp85_p90,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,14.79,55.47,98.04,47.50,105.16,167.89,94.80,177.96,...,2432.93,751.69,976.58,1139.29,843.55,1112.96,1297.70,949.12,1246.59,1447.63
1,40.041039,-88.961884,14.71,55.04,97.38,47.11,104.52,166.96,94.39,176.66,...,2430.76,751.58,976.73,1139.12,843.14,1113.03,1297.47,948.72,1246.07,1447.36
2,40.041039,-88.878548,14.60,54.70,96.73,46.80,103.91,166.10,93.90,175.70,...,2429.90,751.60,977.00,1139.19,842.73,1113.26,1297.56,948.69,1245.65,1447.23
3,40.041039,-88.795219,14.29,53.96,95.60,46.16,102.68,164.46,92.78,173.90,...,2429.09,749.92,975.50,1137.34,840.50,1111.41,1295.62,946.56,1242.96,1445.07
4,40.041039,-88.711891,14.34,54.06,95.70,46.18,102.68,164.39,92.55,173.82,...,2430.25,752.34,978.33,1139.99,842.57,1113.95,1298.11,948.27,1244.80,1447.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,430.46,0.03,0.13,1.06,1.84,3.85,12.36,10.02,18.95,44.66
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,405.32,0.00,0.03,0.45,1.14,2.44,8.99,7.93,14.90,37.28
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,422.37,0.01,0.06,0.72,1.57,3.30,10.83,9.29,17.27,41.88
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,428.35,0.02,0.08,0.81,1.74,3.57,11.38,9.77,17.98,43.50


In [15]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_seasonal.feather")

# Merge df45 and 85

# Extract hist

In [16]:
dfse852 = dfse85.drop(columns=["winter_hist_rcp85_p10","winter_hist_rcp85_p50", "winter_hist_rcp85_p90", 
                               "spring_hist_rcp85_p10", "spring_hist_rcp85_p50", "spring_hist_rcp85_p90", 
                               "summer_hist_rcp85_p10","summer_hist_rcp85_p50", "summer_hist_rcp85_p90", 
                               "fall_hist_rcp85_p10","fall_hist_rcp85_p50", "fall_hist_rcp85_p90"])
dfse852


,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,summer_t2050_rcp85_p50,summer_t2080_rcp85_p50,summer_t2050_rcp85_p90,summer_t2080_rcp85_p90,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,55.47,98.04,105.16,167.89,177.96,282.28,811.24,921.28,...,2075.44,2274.91,2216.34,2432.93,976.58,1139.29,1112.96,1297.70,1246.59,1447.63
1,40.041039,-88.961884,55.04,97.38,104.52,166.96,176.66,280.82,809.38,919.05,...,2074.07,2273.28,2214.32,2430.76,976.73,1139.12,1113.03,1297.47,1246.07,1447.36
2,40.041039,-88.878548,54.70,96.73,103.91,166.10,175.70,279.52,808.26,917.79,...,2074.14,2273.38,2213.46,2429.90,977.00,1139.19,1113.26,1297.56,1245.65,1447.23
3,40.041039,-88.795219,53.96,95.60,102.68,164.46,173.90,277.33,806.04,915.08,...,2074.17,2273.34,2212.43,2429.09,975.50,1137.34,1111.41,1295.62,1242.96,1445.07
4,40.041039,-88.711891,54.06,95.70,102.68,164.39,173.82,277.28,807.73,916.72,...,2076.57,2275.36,2213.53,2430.25,978.33,1139.99,1113.95,1298.11,1244.80,1447.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,2.54,0.00,0.00,...,182.11,241.62,333.76,430.46,0.13,1.06,3.85,12.36,18.95,44.66
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,2.02,0.00,0.00,...,163.45,218.27,310.78,405.32,0.03,0.45,2.44,8.99,14.90,37.28
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,2.04,0.00,0.00,...,174.84,232.68,326.88,422.37,0.06,0.72,3.30,10.83,17.27,41.88
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,1.87,0.00,0.00,...,178.57,237.34,332.20,428.35,0.08,0.81,3.57,11.38,17.98,43.50


## Getting historic mean for each season

In [17]:
### Function
def mean_season(season, dfse45, dfse85):
    dfw45 = dfse45[["lat", "lon", season+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", season+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[season+"_hist_p50"] = round((dfsew[season+"_hist_rcp45_p50"]+ dfsew[season+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", season+"_hist_p50"]]
    return (dftgwh3)


### Winter historic mean

In [18]:
dftgwh = mean_season("winter", dfse45, dfse85)
dftgwh

,lat,lon,winter_hist_p50
0,40.041039,-89.045212,47.88
1,40.041039,-88.961884,47.35
2,40.041039,-88.878548,46.99
3,40.041039,-88.795219,46.33
4,40.041039,-88.711891,46.24
...,...,...,...
76201,66.623306,-61.629639,0.00
76202,66.623306,-61.546310,0.00
76203,66.623306,-61.462978,0.00
76204,66.623306,-61.379650,0.00


### Spring historic mean

In [19]:
dftsph = mean_season("spring", dfse45, dfse85)
dftsph

,lat,lon,spring_hist_p50
0,40.041039,-89.045212,738.88
1,40.041039,-88.961884,736.66
2,40.041039,-88.878548,734.94
3,40.041039,-88.795219,732.14
4,40.041039,-88.711891,733.20
...,...,...,...
76201,66.623306,-61.629639,0.00
76202,66.623306,-61.546310,0.00
76203,66.623306,-61.462978,0.00
76204,66.623306,-61.379650,0.00


### Summer historic mean

In [20]:
dftsuh = mean_season("summer", dfse45, dfse85)
dftsuh

,lat,lon,summer_hist_p50
0,40.041039,-89.045212,1794.64
1,40.041039,-88.961884,1793.05
2,40.041039,-88.878548,1792.82
3,40.041039,-88.795219,1792.62
4,40.041039,-88.711891,1794.74
...,...,...,...
76201,66.623306,-61.629639,136.92
76202,66.623306,-61.546310,120.24
76203,66.623306,-61.462978,130.17
76204,66.623306,-61.379650,132.96


### Fall historic mean

In [21]:
dftfah = mean_season("fall", dfse45, dfse85)
dftfah

,lat,lon,fall_hist_p50
0,40.041039,-89.045212,843.16
1,40.041039,-88.961884,842.71
2,40.041039,-88.878548,842.41
3,40.041039,-88.795219,840.26
4,40.041039,-88.711891,842.32
...,...,...,...
76201,66.623306,-61.629639,1.74
76202,66.623306,-61.546310,1.08
76203,66.623306,-61.462978,1.50
76204,66.623306,-61.379650,1.66


# Merge means with the 45 and 85 DF

In [22]:
dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [23]:
dftgseall

,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90,fall_hist_p50,summer_hist_p50,spring_hist_p50,winter_hist_p50
0,40.041039,-89.045212,55.47,98.04,105.16,167.89,177.96,282.28,811.24,921.28,...,910.20,925.41,1032.10,1085.45,1164.08,1230.76,843.16,1794.64,738.88,47.88
1,40.041039,-88.961884,55.04,97.38,104.52,166.96,176.66,280.82,809.38,919.05,...,910.01,925.28,1032.31,1085.44,1164.24,1230.68,842.71,1793.05,736.66,47.35
2,40.041039,-88.878548,54.70,96.73,103.91,166.10,175.70,279.52,808.26,917.79,...,909.96,925.25,1032.50,1085.60,1164.42,1230.74,842.41,1792.82,734.94,46.99
3,40.041039,-88.795219,53.96,95.60,102.68,164.46,173.90,277.33,806.04,915.08,...,908.31,923.69,1030.48,1083.51,1162.31,1228.63,840.26,1792.62,732.14,46.33
4,40.041039,-88.711891,54.06,95.70,102.68,164.39,173.82,277.28,807.73,916.72,...,911.11,926.38,1032.70,1086.22,1164.37,1230.98,842.32,1794.74,733.20,46.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,2.54,0.00,0.00,...,0.04,0.02,2.58,2.88,14.86,13.53,1.74,136.92,0.00,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,2.02,0.00,0.00,...,0.02,0.00,1.41,1.74,11.81,10.48,1.08,120.24,0.00,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,2.04,0.00,0.00,...,0.03,0.00,2.10,2.43,13.78,12.36,1.50,130.17,0.00,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,1.87,0.00,0.00,...,0.04,0.01,2.33,2.65,14.40,12.93,1.66,132.96,0.00,0.00


In [25]:
dftgseall.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_seasonal.feather")

In [26]:
#dftgseall = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [27]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [28]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [29]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [30]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_DJC_seasonal.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_DJC_seasonal.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_DJC_seasonal.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
